### Simple parametric log spiral example
In theory, a logarithmic spiral pattern produces a cone/anticone surface (the Gaussian curvature is a delta function, vanishing everywhere away from the central singularity).
The total curvature is controlled by the spiral angle `alpha`.
In practice, the physical system smoothes out the curvature singularity.

In [ ]:
import sys; sys.path.append('..')
import inflation, numpy as np, importlib
import fd_validation, visualization, parametric_pillows, wall_generation
import parallelism, benchmark
from numpy.linalg import norm

In [ ]:
parallelism.set_max_num_tbb_threads(4)

In [ ]:
from ipywidgets import interactive, widgets
def plotForAlpha(alpha): visualization.plot_line_segments(*parametric_pillows.logSpiralPlot(alpha=alpha, edgeLength=0.02, minDist=0.05, margin=.01))
iplot = interactive(plotForAlpha, alpha = widgets.FloatSlider(min=1, max=90, value=70, step=1))
iplot.children[-1].layout.height = '500px'
display(iplot)

In [ ]:
importlib.reload(parametric_pillows)
importlib.reload(visualization)
m, fuseMarkers, fuseEdges = wall_generation.triangulate_channel_walls(*parametric_pillows.logSpiralPlot(
                alpha=iplot.children[0].value, edgeLength=0.02, minDist=0.075, margin=0.025), 0.0002)
visualization.plot_2d_mesh(m, pointList=np.where(np.array(fuseMarkers) == 1)[0], width=12, height=12, )

In [ ]:
isheet = inflation.InflatableSheet(m, np.array(fuseMarkers) != 0)

In [ ]:
import py_newton_optimizer
isheet.setUseTensionFieldEnergy(True)
niter = 5000
iterations_per_output = 10
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.useIdentityMetric = True
opts.beta = 1e-4
opts.gradTol = 1e-10
opts.niter = iterations_per_output

In [ ]:
from tri_mesh_viewer import TriMeshViewer
viewer = TriMeshViewer(isheet.visualizationMesh(), width=1024, height=768)
viewer.showWireframe()
viewer.show()

In [ ]:
import time
isheet.pressure = 50
benchmark.reset()
for step in range(int(niter / iterations_per_output)):
    cr = inflation.inflation_newton(isheet, isheet.rigidMotionPinVars, opts)
    if cr.numIters() < iterations_per_output: break
    #isheet.writeDebugMesh('log_spiral_alpha_20/inflation_tf_step_{}.msh'.format(step))
    viewer.update(False, isheet.visualizationMesh())
    time.sleep(0.05) # Allow some mesh synchronization time for pythreejs
benchmark.report()

In [ ]:
isheet.tensionStateHistogram()